<a href="https://colab.research.google.com/github/mayarj/model_finetune_for_verigen/blob/main/modelFinetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning AI Model for Verilog
This notebook is part of the project aimed at fine-tuning an AI model for Verilog code generation.

## Acknowledgments
This work uses the dataset from the paper "PyraNet: A Neural Network for Verilog Code Generation" (2024). The dataset can be accessed at [bnadimi/PyraNet-Verilog](https://huggingface.co/datasets/bnadimi/PyraNet-Verilog).



In [ ]:
!pip install datasets
from datasets import load_dataset
custom_cache_dir = "./custom_cache_dir"
dataset = load_dataset("bnadimi/PyraNet-Verilog" , cache_dir=custom_cache_dir)
example_instance = dataset['train'][0:5]
print(example_instance)
print("\n")
print (dataset['train']["code"][0:5])

print("\n")
print (dataset['train'][0:5]["code"])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

PyraNetOnVeriBest.csv:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

Generating train split:   0%|          | 0/692238 [00:00<?, ? examples/s]

{'code': ["module VCC (output V);\n   assign V = 1'b1;\nendmodule", 'module pg(\n  input  wire i_a,\n  input  wire i_b,\n  output wire o_p,\n  output wire o_g\n);\n\nassign o_p = i_a ^ i_b;\nassign o_g = i_a & i_b;\n\nendmodule', 'module NOT16(\n    input [15:0] in,\n    output [15:0] out\n);\n\nassign out = ~in;\n\nendmodule', 'module mod(\n\tinput in,\n\toutput out\n);\n\nassign out = in;\n\nendmodule', 'module MyNot(a, z);\n\ninput a;\noutput z;\n\nassign z = ~a;\n\nendmodule'], 'description': ['{"description": "The Verilog code defines a module named `VCC` that outputs a constant high logic level (1). The output `V` is always set to 1.", "rank": "20", "complexity": "Basic", "compile_status": "No error!", "compile_results": ""}', '{"description": "The Verilog code implements a hardware module for generating the propagate (`o_p`) and generate (`o_g`) signals in a binary adder. It computes `o_p` as the XOR of the inputs (`i_a` and `i_b`) and `o_g` as the AND of the same inputs.", "ran

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import time
import csv
import os
import json
import gc
from tqdm import tqdm
def save_dataset_to_csv_by_complexity(dataset):
    """
    Efficiently saves 700,000+ entry dataset to CSV files by complexity level.
    Provides progress updates and maintains low memory usage.
    """
    # Create output directory
    output_dir = "/content/drive/MyDrive/Verilog_dataset/complexity_datasets"
    os.makedirs(output_dir, exist_ok=True)

    # Track file handles and writers
    complexity_writers = {}
    file_handles = {}

    def init_writer(complexity):
        """Initialize CSV writer for a complexity level"""
        if complexity not in complexity_writers:
            csv_path = os.path.join(output_dir, f"{complexity}.csv")
            fh = open(csv_path, 'w', newline='', encoding='utf-8')
            writer = csv.DictWriter(fh, fieldnames=[
                'code', 'description', 'rank', 'complexity',
                'compile_status', 'compile_results'
            ])
            writer.writeheader()
            complexity_writers[complexity] = writer
            file_handles[complexity] = fh

    total_entries = len(dataset)
    print(f"Processing large dataset ({total_entries:,} entries)...")
    start_time = time.time()
    last_print = start_time

    # Process with manual iteration for better memory control
    for i in tqdm(range(total_entries), desc="Processing", unit="entries"):
        try:
            # Access entry directly (memory efficient)
            entry = dataset[i]

            # Parse the data
            code = entry['code']
            desc_data = entry['description'].replace('\\"', '"')
            desc_dict = json.loads(desc_data)

            # Get fields with defaults
            complexity = desc_dict.get('complexity', 'unknown')
            init_writer(complexity)

            complexity_writers[complexity].writerow({
                'code': code,
                'description': desc_dict.get('description', ''),
                'rank': desc_dict.get('rank', 'unknown'),
                'complexity': complexity,
                'compile_status': desc_dict.get('compile_status', 'unknown'),
                'compile_results': desc_dict.get('compile_results', '')
            })

            # Progress update every 1000 rows with timing info
            if (i + 1) % 1000 == 0:
                current_time = time.time()
                elapsed = current_time - start_time
                items_per_sec = (i + 1) / elapsed
                remaining = (total_entries - (i + 1)) / items_per_sec

                print(
                    f"Processed {i + 1:,} of {total_entries:,} | "
                    f"Speed: {items_per_sec:.1f} entries/sec | "
                    f"ETA: {remaining/60:.1f} minutes remaining"
                )
                last_print = current_time

            # Memory management
            if i % 10000 == 0:
                gc.collect()

        except json.JSONDecodeError as e:
            print(f"\nJSON Error at index {i}: {e}...")
            break
        except Exception as e:
            print(f"\nError at index {i}: {e}...")
            break

    # Cleanup
    for fh in file_handles.values():
        fh.close()

    total_time = (time.time() - start_time)/60
    print(f"\nCompleted processing {total_entries:,} entries in {total_time:.1f} minutes")

save_dataset_to_csv_by_complexity(dataset['train'])

Processing large dataset (692,238 entries)...


Processing:   0%|          | 1368/692238 [00:00<03:19, 3465.88entries/s]

Processed 1,000 of 692,238 | Speed: 2921.7 entries/sec | ETA: 3.9 minutes remaining


Processing:   0%|          | 2656/692238 [00:00<03:21, 3428.93entries/s]

Processed 2,000 of 692,238 | Speed: 2862.0 entries/sec | ETA: 4.0 minutes remaining
Processed 3,000 of 692,238 | Speed: 3361.2 entries/sec | ETA: 3.4 minutes remaining


...

Processed 690,000 of 692,238 | Speed: 3282.2 entries/sec | ETA: 0.0 minutes remaining


Processing: 100%|█████████▉| 691293/692238 [03:31<00:00, 1577.96entries/s]

Processed 691,000 of 692,238 | Speed: 3275.6 entries/sec | ETA: 0.0 minutes remaining


Processing: 100%|██████████| 692238/692238 [03:31<00:00, 3270.09entries/s]

Processed 692,000 of 692,238 | Speed: 3271.3 entries/sec | ETA: 0.0 minutes remaining

Completed processing 692,238 entries in 3.5 minutes


In [20]:
import os
import csv
import shutil
from pathlib import Path

def process_csv_files(source_dir, output_dir):
    """
    Process CSV files to remove rows with empty or "0" description,
    and save cleaned files to output directory while preserving originals.
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(source_dir):
        if not filename.endswith('.csv'):
            continue

        source_path = os.path.join(source_dir, filename)
        output_path = os.path.join(output_dir, filename)

        print(f"Processing {filename}...")

        try:
            with open(source_path, 'r', newline='', encoding='utf-8') as infile, \
                 open(output_path, 'w', newline='', encoding='utf-8') as outfile:

                reader = csv.DictReader(infile)
                fieldnames = reader.fieldnames

                # Verify 'description' field exists
                if 'description' not in fieldnames:
                    print(f"⚠️ 'description' column not found in {filename} - skipping")
                    continue

                writer = csv.DictWriter(outfile, fieldnames=fieldnames)
                writer.writeheader()

                removed_count = 0
                kept_count = 0

                for row in reader:
                    description = str(row['description']).strip()
                    if not description or description == '0':
                        removed_count += 1
                    else:
                        writer.writerow(row)
                        kept_count += 1

                print(f"✅ Finished {filename}:")
                print(f"   - Kept {kept_count} rows")
                print(f"   - Removed {removed_count} rows")
                print(f"   - Cleaned file saved to {output_path}")

        except Exception as e:
            print(f"❌ Error processing {filename}: {str(e)}")
            # Remove the partial output file if there was an error
            if os.path.exists(output_path):
                os.remove(output_path)

if __name__ == "__main__":
    print("CSV Cleaner - Remove rows with empty/0 description")

    # Set your directories here
    source_directory = "/content/drive/MyDrive/Verilog_dataset/complexity_datasets"  # Replace with your source directory
    output_directory = "/content/drive/MyDrive/Verilog_dataset/seperated_dataset"   # Replace with your output directory

    if not os.path.exists(source_directory):
        print(f"❌ Source directory does not exist: {source_directory}")
        exit(1)

    process_csv_files(source_directory, output_directory)
    print("\n🎉 All files processed! Original files remain unchanged.")

CSV Cleaner - Remove rows with empty/0 description
Processing Basic.csv...
✅ Finished Basic.csv:
   - Kept 24633 rows
   - Removed 102874 rows
   - Cleaned file saved to /content/drive/MyDrive/Verilog_dataset/seperated_dataset/Basic.csv
Processing Intermediate.csv...
✅ Finished Intermediate.csv:
   - Kept 123176 rows
   - Removed 64720 rows
   - Cleaned file saved to /content/drive/MyDrive/Verilog_dataset/seperated_dataset/Intermediate.csv
Processing Expert.csv...
✅ Finished Expert.csv:
   - Kept 145125 rows
   - Removed 4 rows
   - Cleaned file saved to /content/drive/MyDrive/Verilog_dataset/seperated_dataset/Expert.csv
Processing Advanced.csv...
✅ Finished Advanced.csv:
   - Kept 228344 rows
   - Removed 3362 rows
   - Cleaned file saved to /content/drive/MyDrive/Verilog_dataset/seperated_dataset/Advanced.csv

🎉 All files processed! Original files remain unchanged.


In [22]:
import os
import csv
import random
import tempfile
import sys
from pathlib import Path
import shutil

def increase_csv_field_size_limit():
    """Increase CSV field size limit to handle large fields."""
    max_int = sys.maxsize
    while True:
        try:
            csv.field_size_limit(max_int)
            break
        except OverflowError:
            max_int = int(max_int / 2)

def process_large_csv(source_dir, dest_dir,oregenal_dir, sample_size=10):
    """Process large CSV files with large field support."""
    increase_csv_field_size_limit()
    os.makedirs(dest_dir, exist_ok=True)
    os.makedirs(oregenal_dir, exist_ok=True)
    for filename in os.listdir(source_dir):
        if not filename.endswith('.csv'):
            continue

        source_path = os.path.join(source_dir, filename)
        dest_path = os.path.join(dest_dir, f"fortest_{filename}")
        original_backup_path = os.path.join(oregenal_dir, f"original_{filename}")

        print(f"⚙️ Processing {filename}...")

        # First pass: count error rows and their positions
        error_positions = []
        try:
            with open(source_path, 'r', newline='', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                for i, row in enumerate(reader):
                    if row.get('compile_status').strip().lower() == 'no error!':
                        error_positions.append(i)
        except csv.Error as e:
            print(f"⚠️ CSV parsing error in {filename}: {str(e)}")
            continue

        if not error_positions:
            print(f"⚠️ No error rows in {filename}")
            continue

        # Select random rows to remove
        selected_positions = set(random.sample(error_positions, min(sample_size, len(error_positions))))

        # Second pass: process file in streaming fashion
        temp_file = tempfile.NamedTemporaryFile(mode='w', delete=False, newline='', encoding='utf-8')
        removed_count = 0

        try:
            with open(source_path, 'r', newline='', encoding='utf-8') as infile, \
                 open(dest_path, 'w', newline='', encoding='utf-8') as outfile:

                reader = csv.DictReader(infile)
                writer_keep = csv.DictWriter(temp_file, fieldnames=reader.fieldnames)
                writer_remove = csv.DictWriter(outfile, fieldnames=reader.fieldnames)

                writer_keep.writeheader()
                writer_remove.writeheader()

                for i, row in enumerate(reader):
                    if i in selected_positions:
                        writer_remove.writerow(row)
                        removed_count += 1
                    else:
                        writer_keep.writerow(row)
        except Exception as e:
            print(f"❌ Error processing {filename}: {str(e)}")
            temp_file.close()
            os.unlink(temp_file.name)
            continue

        temp_file.close()



        shutil.move(temp_file.name, original_backup_path )
        print(f"✅ Processed {filename}:")
        print(f"  - Removed {removed_count} rows to: {os.path.basename(dest_path)}")
        print(f"  - Original saved as: original_{filename}")
        print(f"  - Filtered version saved in original location")

print("📂 Large CSV Processor (2GB+ support)")
oregenal_directory  = "/content/drive/MyDrive/Verilog_dataset/complexity_datasets"
dest_directory = "/content/drive/MyDrive/Verilog_dataset/Test_dataset"
source_directory = "/content/drive/MyDrive/Verilog_dataset/seperated_dataset"

if not os.path.exists(source_directory):
    print("❌ Source directory doesn't exist!")
    exit(1)

process_large_csv(source_directory, dest_directory,oregenal_directory )
print("\n🎉 Processing complete!")

📂 Large CSV Processor (2GB+ support)
⚙️ Processing Basic.csv...
✅ Processed Basic.csv:
  - Removed 10 rows to: fortest_Basic.csv
  - Original saved as: original_Basic.csv
  - Filtered version saved in original location
⚙️ Processing Intermediate.csv...
✅ Processed Intermediate.csv:
  - Removed 10 rows to: fortest_Intermediate.csv
  - Original saved as: original_Intermediate.csv
  - Filtered version saved in original location
⚙️ Processing Expert.csv...
✅ Processed Expert.csv:
  - Removed 10 rows to: fortest_Expert.csv
  - Original saved as: original_Expert.csv
  - Filtered version saved in original location
⚙️ Processing Advanced.csv...
✅ Processed Advanced.csv:
  - Removed 10 rows to: fortest_Advanced.csv
  - Original saved as: original_Advanced.csv
  - Filtered version saved in original location

🎉 Processing complete!
